# Обучение ансамблей

## Задача

Обучить объединяющую надстройку над обученными моделя.

## Данные

Классы с архитектурами ансамблей и данные для обучения и теста взяты из файла *dataset_and_models.py*

## Расчёты

In [1]:
import torch
from torch import nn, optim
from dataset_and_models import ENSEMBLE
from dataset_and_models import w2v_data_train, w2v_data_test
from dataset_and_models import fasttext_data_train, fasttext_data_test
from sklearn.metrics import f1_score, accuracy_score
from tqdm.notebook import tqdm

In [2]:
# установка устройва для расчётов
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
def train_ensemble(
        data_train,
        data_test,
        name_model,
        device=DEVICE,
        lr=0.0005,
        ecophs=5
):
    """
    Функция обучения ансамбля
    :param data_train: данные обучения
    :param data_test: данные теста
    :param name_model: имя модели ансамбля
    :param device: устройво для основных расчётов
    :param lr: скорость обучения
    :param ecophs: количество эпох обучения
    """
    # создание объекта ансамбля
    model = ENSEMBLE(name=name_model).to(device)
    
    # определение функции ошибки и оптимизатора
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # хранит лучший F1-score
    best_f1_score = 0

    # цикл обучения
    for epoch in range(ecophs):

        model.to(device)
        model.train()

        progress_bar = tqdm(data_train, desc=f'Эпоха {epoch + 1}')

        # обучение модели в пределах одной эпохи
        for x, y in progress_bar:

            x, y = x.to(device), y.to(device)
            y_hat = model(x)

            optimizer.zero_grad()
            loss = loss_fn(y_hat, y)
            loss.backward()
            optimizer.step()

        # оценка ансамбля после эпохи обучения    
        model.eval()
        
        output_model = torch.Tensor([]).to(device)
        output_y = torch.Tensor([]).to(device)
        
        with torch.no_grad():
            for x, y in data_test:
                x, y = x.to(device), y.to(device)
                output_model = torch.concat([output_model, model(x).argmax(axis=1)])
                    
                output_y = torch.concat([output_y, y])
        
        output_y = output_y.to('cpu')
        output_model = output_model.to('cpu')
        f1 = f1_score(
            y_true=output_y.numpy(),
            y_pred=output_model.numpy(),
            average='weighted'
        )
        
        # вывод информации об оценках ансамбля по итогу эпохи
        print('F1-score (weighted):\t', f1)
        print('Точность:\t\t', accuracy_score(y_true=output_y.numpy(),
                                              y_pred=output_model.numpy()))
        
        # сохранение ансамбля с наилучшим F1-score
        if best_f1_score < f1:
            
            best_f1_score = f1
            torch.save(model, f'models/{name_model}.pt')

            print('===Модель сохранена===')

### Обучение ансамбля состоящего из CNN и LSTM

In [4]:
# w2v
train_ensemble(
    data_train=w2v_data_train,
    data_test=w2v_data_test,
    name_model='w2v_cnn_and_lstm_ensemble'
)

Эпоха 1:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7531668786615816
Точность:		 0.7651158024185284
===Модель сохранена===


Эпоха 2:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7560784999414428
Точность:		 0.7612215617954499
===Модель сохранена===


Эпоха 3:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7589009547920693
Точность:		 0.7630662020905923
===Модель сохранена===


Эпоха 4:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7591523318073435
Точность:		 0.7622463619594179
===Модель сохранена===


Эпоха 5:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.758277330430149
Точность:		 0.7616314818610371


In [5]:
# fasttext
train_ensemble(
    data_train=fasttext_data_train,
    data_test=fasttext_data_test,
    name_model='fasttext_cnn_and_lstm_ensemble'
)

Эпоха 1:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7367301038441715
Точность:		 0.7610166017626563
===Модель сохранена===


Эпоха 2:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7498853623588304
Точность:		 0.7542529206804673
===Модель сохранена===


Эпоха 3:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7513329193956761
Точность:		 0.7530231604837057
===Модель сохранена===


Эпоха 4:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7517870032611008
Точность:		 0.7554826808772289
===Модель сохранена===


Эпоха 5:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7512876370404143
Точность:		 0.7534330805492929


### Обучение ансамбля состоящего из LSTM_CNN моделей

In [6]:
# w2v
train_ensemble(
    data_train=w2v_data_train,
    data_test=w2v_data_test,
    name_model='w2v_ensemble_of_lstm_cnn'
)

Эпоха 1:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7636135979098875
Точность:		 0.7661406025824964
===Модель сохранена===


Эпоха 2:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7629252599892978
Точность:		 0.7628612420577987


Эпоха 3:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.763615944196096
Точность:		 0.7640910022545604
===Модель сохранена===


Эпоха 4:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7624271416805869
Точность:		 0.7628612420577987


Эпоха 5:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7612316499132574
Точность:		 0.7622463619594179


In [7]:
# fasttext
train_ensemble(
    data_train=fasttext_data_train,
    data_test=fasttext_data_test,
    name_model='fasttext_ensemble_of_lstm_cnn'
)

Эпоха 1:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7592468219477383
Точность:		 0.7700348432055749
===Модель сохранена===


Эпоха 2:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7621043974930682
Точность:		 0.7704447632711621
===Модель сохранена===


Эпоха 3:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7616600506907638
Точность:		 0.7702398032383685


Эпоха 4:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7597710502795881
Точность:		 0.7688050830088132


Эпоха 5:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7607508655694215
Точность:		 0.7716745234679238


## Результаты

Были обучены и сохранены надстройки над ансаблями на текстах, обработанных Word2Vec и FastText.